# Run PEST with BeoPEST

### This should be super easy...

In [4]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu
from sm_pst_pkgs.sm_pst_utils import execute_beopest, execute_workers
from sm_pst_pkgs.sm_pst_stats import create_param_unc

# 1. Specify a path to your master

In [5]:
wd = "E:/okvg_pp/okvg_091120_pest"

In [6]:
os.chdir(wd)

# 2. Initial run for adjusting weights

In [4]:
os.system("start cmd /k i64pest {0}".format("okvg_pest2.pst"))

0

# 3. Reweight

In [7]:
casename = "okvg_pest2"
pestoutfile = "okvg_pest2_rw.pst"
contrib = 2.32445E+08

In [12]:
1.14414*1.5

1.7162099999999998

In [5]:
os.system("start cmd /k i64pwtadj1 {0} {1} {2}".format(casename, pestoutfile, contrib))

0

Go check the pst file

#### Calculate Jacobian Matrix

# 4. Run Parallel PEST with BeoPEST

In [ ]:
execute_beopest()

In [6]:
execute_beopest(wd, pestoutfile, num_workers=1, reuse_workers=True, copy_files=['model.in', 'model.in.tpl'])

# 5. Run or add workers separately

In [4]:
execute_workers?

Signature:
execute_workers(
    worker_rep,
    pst,
    host,
    num_workers=None,
    start_id=None,
    worker_root='..',
    port=4005,
    reuse_workers=None,
    copy_files=None,
)
Docstring:
[summary]

Args:
    worker_rep ([type]): [description]
    pst ([type]): [description]
    host ([type]): [description]
    num_workers ([type], optional): [description]. Defaults to None.
    start_id ([type], optional): [description]. Defaults to None.
    worker_root (str, optional): [description]. Defaults to '..'.
    port (int, optional): [description]. Defaults to 4005.

Raises:
    Exception: [description]
    Exception: [description]
    Exception: [description]
    Exception: [description]
File:      e:\okvg_pp\sm_pst_pkgs\sm_pst_utils.py
Type:      function


In [8]:
execute_workers(
    wd, "okvg_pest2_rw.pst", "172.20.1.9", num_workers=8, start_id=12,
    worker_root='..', port=4005,
    # reuse_workers=True,
    # copy_files=['model.in', 'model.in.tpl']
    )





#### Statistics - Create the "param.unc" file

- An easy way to formulate a list of prior parameter standard deviations (this being one option for filling a parameter uncertainty file) is to divide the difference between these bounds (or their logs for parameters that are log-transformed) by 4.0. 
- This procedure relies on the the implicit assumption that parameters are (log)normally distributed and that their bounds span their 95% confidence intervals.

In [5]:
create_param_unc?

Signature: create_param_unc(pst_file, unc_file=None, sampl_n=None)
Docstring:
create a parameter uncertainty file from an existing *.pst file

Args:
    - pst_file (`str`): path and name of existing *.pst file
    - unc_file (`str`): name of parameter uncertainty file
                        If `None`, then `param.unc` is used.
                        Defult is `None`.
    - sampl_n ('int'): sample number from normal distribution
                        If `None`, then `1000` is used.
                        Defult is `None`.
Returns:
    `pandas.DataFrame`: a dataframe of log standard deviation for each parameter
    `param.unc file`

Example:
    sm_pst_stats.create_param_unc('my.pst', 'my.unc', 2000)
File:      e:\qsm_300_zon_p\sm_pst_pkgs\sm_pst_stats.py
Type:      function


In [12]:
create_param_unc("qsm_reweight.pst")

param.unc file has been created...


,parlbnd,parubnd,mu,sigma,lower_95,upper_95,log_sd
parnme,,,,,,,
ed,1.0000000000e+00,1.0000000000e+02,50.500000,24.750000,1.932049,99.067951,0.427479
kc,2.5924320000e-03,2.5924320000e+01,12.963456,6.480432,0.246636,25.680276,0.504386
kdfdc,2.0347200000e-03,2.0347200000e+01,10.174617,5.086291,0.193577,20.155658,0.504386
kked,5.1840000000e-03,5.1840000000e+01,25.922592,12.958704,0.493190,51.351994,0.504386
kms,1.2960000000e-03,1.2960000000e+01,6.480648,3.239676,0.123297,12.837999,0.504386
kpw,2.5924320000e-03,2.5924320000e+01,12.963456,6.480432,0.246636,25.680276,0.504386
qal,2.9600000000e+01,1.0000000000e+03,514.800000,242.600000,38.735962,990.864038,0.351975
qt,5.9600000000e+01,1.5000000000e+03,779.800000,360.100000,73.160840,1486.439160,0.326967
riv_bot,3.5000000000e-01,6.5000000000e-01,0.500000,0.075000,0.352824,0.647176,0.065866


In [16]:
test = create_param_unc("qsm_reweight.pst")

param.unc file has been created...


#### Calibration with SVD

In [5]:
execute_beopest(wd, "qsm_svd.pst", 10)

#### Calibration with SVD-Reg

In [4]:
execute_beopest(wd, "svd-reg.pst", 15)

#### Calibration

In [5]:
execute_beopest(wd, "qsm_reweight.pst", 10)

In [6]:
import socket
socket.gethostname()

'ARSTXTEM4119734'